In [1]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import urllib.request
import time
import io
import sys
import asyncio
import re
from selenium import webdriver
from selenium.webdriver.support.ui import Select
import chromedriver_autoinstaller
path = chromedriver_autoinstaller.install()
from parsel import Selector
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

import winsound as ws

#exec_path = 'D:\\xref\\scrape\\webdriver\\chromedriver'
pd.set_option('display.max_colwidth', None)
driver = webdriver.Chrome(executable_path = path)

import warnings
warnings.filterwarnings("ignore")

# STARTUP_DELAY = 1
# NEXT_PAGE_DELAY =1
# WAIT_AFTER_ELEMENT_LOADED = 1
# DELAY_TO_CHECK_IF_MEW_ELEMENT_IS_VISIBLE =1
# LOAD_SPECIFIC_RESULT_DELAY = 1
# BACK_TO_MAIN_LIST_DELAY = 1

In [2]:
raw_text = ['PULASKI COUNTY AR']#, 'Saint Charls County MO']
replace_dict = {"AR": "ARKANSAS"}
for key, value in replace_dict.items():
    raw_text = [item.replace(key, value) for item in raw_text]
    
search_string_list = ["Physical Thrapy Clinic"]

In [3]:
print("Raw text:\n", raw_text)

print("search string:\n", search_string_list)

Raw text:
 ['PULASKI COUNTY ARKANSAS']
search string:
 ['Physical Thrapy Clinic']


In [4]:
#Obtain information for Businesses from google map
df_level_1 = pd.DataFrame(columns=["LocName","LocHREF"])

N = 10
idx = 0

for prefix in raw_text:
    t1 = pd.Timestamp.now()
    df_level_1.to_pickle('Physical_Therapy_MO_level_1_scrape_intermediate.pkl')
    prefix = prefix.replace(' ',"+")
    print(prefix)
    for search_string in search_string_list:
        search_string = search_string.replace(' ',"+")
        print(search_string)

        url_local = search_string + '+' + prefix
        url_local = 'https://www.google.com/maps/search/' + url_local
        print(url_local)
        driver.get(url_local)
        driver.maximize_window()
        time.sleep(1.5)

        driver.execute_script("document.body.style.zoom='zoom 50%'") #does not work!!

        keep_looping = True
        page_number = 0
        while page_number <= N:
            action = webdriver.ActionChains(driver)
            soup = BeautifulSoup(driver.page_source, 'lxml')
    #         print(soup)
            
            for button_tmp in soup.find_all('button'):
                try:

                    if button_tmp['aria-label'] == ' Next page ':
                        button_id = button_tmp['id']

                except:
                    pass

        
            python_button = driver.find_element_by_id(button_id)
            action.move_to_element(python_button)
            action.perform()
            action.move_by_offset(35, -50)
            action.double_click()
            action.perform()
            time.sleep(.25)
            action.double_click()
            action.perform()
            time.sleep(.25)
            action.double_click()
            action.perform()
            time.sleep(.25)
            action.double_click()
            action.perform()
            time.sleep(.25)
            action.double_click()
            action.perform()
            time.sleep(.25)

            page_content = driver.page_source
    #         print(page_content)


            response = Selector(page_content)

            for el in response.xpath('//div[contains(@aria-label, "Results for")]/div/div[./a]'):
                LocHREF = el.xpath('./a/@href').extract_first('')
                df_level_1.loc[idx,"LocHREF"] = LocHREF

                LocName= el.xpath('./a/@aria-label').extract_first('')
                df_level_1.loc[idx,"LocName"] = LocName

                Business = el.xpath('//*[@id="QA0Szd"]/div/div/div[1]/div[2]/div/div[1]/div/div/div[2]/div[1]/div[3]/div/div[2]/div[2]/div[1]/div/div/div/div[4]/div[1]/span[1]/jsl/span[2]').extract_first('')
                Business = re.search(r'<span jstcache="\d{3}">(.*)<\/span>',Business).group(1)
                df_level_1.loc[idx,"Business"] = Business
                
                df_level_1.loc[idx,"raw text"] = prefix
                df_level_1.loc[idx,"search string"] = search_string  
                
                idx += 1

#             if (page_number+1)%10==0:
#                 df_level_1.to_csv("google_map_result_{}_{}_{}.csv".format(prefix,search_string,int(page_number+1)))
#                 df_level_1 = pd.DataFrame(columns=["LocName","LocHREF"])
#                 idx = 0
        
            page_number += 1
        
            if page_number <= N:
                try:
                    python_button = driver.find_element_by_id(button_id)
                    python_button.click()
                    time.sleep(1)
                except:
                    #print('reached end of good buttons')
                    # keep_looping = False
                    pass

        df_level_1.to_csv("google_map_result_{}_{}.csv".format(prefix,search_string),index = False)#,int(page_number)))              
#         driver.quit();
                
df_level_1

PULASKI+COUNTY+ARKANSAS
Physical+Thrapy+Clinic
https://www.google.com/maps/search/Physical+Thrapy+Clinic+PULASKI+COUNTY+ARKANSAS


,LocName,LocHREF,Business,raw text,search string
0,UAMS Health - Physical Therapy and Occupational Therapy Center,https://www.google.com/maps/place/UAMS+Health+-+Physical+Therapy+and+Occupational+Therapy+Center/data=!4m6!3m5!1s0x87d2a4b1aaaaaaab:0xe81e8e3d113da9b6!8m2!3d34.7473395!4d-92.318282!16s%2Fg%2F1vyxjyxq?authuser=0&hl=en&rclk=1,Physical therapy clinic,PULASKI+COUNTY+ARKANSAS,Physical+Thrapy+Clinic
1,MCH Physical Therapy,https://www.google.com/maps/place/MCH+Physical+Therapy/data=!4m6!3m5!1s0x87d2a3fd6cd599fd:0x27c5eaacd1a6be7!8m2!3d34.7599151!4d-92.3801111!16s%2Fg%2F1w456483?authuser=0&hl=en&rclk=1,Physical therapy clinic,PULASKI+COUNTY+ARKANSAS,Physical+Thrapy+Clinic
2,Physical Therapy Advantage LLC,https://www.google.com/maps/place/Physical+Therapy+Advantage+LLC/data=!4m6!3m5!1s0x87cd5f61629df399:0x9220fab6450d8a06!8m2!3d34.8014426!4d-92.4488647!16s%2Fg%2F1th0bzpn?authuser=0&hl=en&rclk=1,Physical therapy clinic,PULASKI+COUNTY+ARKANSAS,Physical+Thrapy+Clinic
3,Pinnacle Physical Therapy,https://www.google.com/maps/place/Pinnacle+Physical+Therapy/data=!4m6!3m5!1s0x87d2a07784ae93bb:0x56fb839ef6e92302!8m2!3d34.797294!4d-92.413297!16s%2Fg%2F11gb3mkx_q?authuser=0&hl=en&rclk=1,Physical therapy clinic,PULASKI+COUNTY+ARKANSAS,Physical+Thrapy+Clinic
4,Blue Rose Physical Therapy,https://www.google.com/maps/place/Blue+Rose+Physical+Therapy/data=!4m6!3m5!1s0xabefe1e4d0190995:0x93c8eacb1abc46e0!8m2!3d34.8017438!4d-92.2551351!16s%2Fg%2F11rrm_w38p?authuser=0&hl=en&rclk=1,Physical therapy clinic,PULASKI+COUNTY+ARKANSAS,Physical+Thrapy+Clinic
...,...,...,...,...,...
164,"Jessica Sanders, PT","https://www.google.com/maps/place/Jessica+Sanders,+PT/data=!4m6!3m5!1s0x87cd7df22b7c7363:0x10065cfbeb0346cd!8m2!3d35.0854363!4d-92.4685146!16s%2Fg%2F1hc2crj5b?authuser=0&hl=en&rclk=1",Orthopedic clinic,PULASKI+COUNTY+ARKANSAS,Physical+Thrapy+Clinic
165,Milestones Pediatric Therapy Services,https://www.google.com/maps/place/Milestones+Pediatric+Therapy+Services/data=!4m6!3m5!1s0x87d2ef154de0288f:0xe09bf50a97e92f6c!8m2!3d34.9526304!4d-92.053232!16s%2Fg%2F11c75t_4cq?authuser=0&hl=en&rclk=1,Orthopedic clinic,PULASKI+COUNTY+ARKANSAS,Physical+Thrapy+Clinic
166,OrthoSurgeons,https://www.google.com/maps/place/OrthoSurgeons/data=!4m6!3m5!1s0x87d2a4808844fcd3:0x564ee64531f64b0c!8m2!3d34.7514216!4d-92.3369286!16s%2Fg%2F1v16qvg6?authuser=0&hl=en&rclk=1,Orthopedic clinic,PULASKI+COUNTY+ARKANSAS,Physical+Thrapy+Clinic
167,"Jessica Sanders, PT","https://www.google.com/maps/place/Jessica+Sanders,+PT/data=!4m6!3m5!1s0x87cd7df22b7c7363:0x10065cfbeb0346cd!8m2!3d35.0854363!4d-92.4685146!16s%2Fg%2F1hc2crj5b?authuser=0&hl=en&rclk=1",Orthopedic clinic,PULASKI+COUNTY+ARKANSAS,Physical+Thrapy+Clinic


In [5]:
data = pd.read_csv("google_map_result_PULASKI+COUNTY+ARKANSAS_Physical+Thrapy+Clinic.csv")
data.head(5)

,LocName,LocHREF,Business,raw text,search string
0,UAMS Health - Physical Therapy and Occupational Therapy Center,https://www.google.com/maps/place/UAMS+Health+-+Physical+Therapy+and+Occupational+Therapy+Center/data=!4m6!3m5!1s0x87d2a4b1aaaaaaab:0xe81e8e3d113da9b6!8m2!3d34.7473395!4d-92.318282!16s%2Fg%2F1vyxjyxq?authuser=0&hl=en&rclk=1,Physical therapy clinic,PULASKI+COUNTY+ARKANSAS,Physical+Thrapy+Clinic
1,MCH Physical Therapy,https://www.google.com/maps/place/MCH+Physical+Therapy/data=!4m6!3m5!1s0x87d2a3fd6cd599fd:0x27c5eaacd1a6be7!8m2!3d34.7599151!4d-92.3801111!16s%2Fg%2F1w456483?authuser=0&hl=en&rclk=1,Physical therapy clinic,PULASKI+COUNTY+ARKANSAS,Physical+Thrapy+Clinic
2,Physical Therapy Advantage LLC,https://www.google.com/maps/place/Physical+Therapy+Advantage+LLC/data=!4m6!3m5!1s0x87cd5f61629df399:0x9220fab6450d8a06!8m2!3d34.8014426!4d-92.4488647!16s%2Fg%2F1th0bzpn?authuser=0&hl=en&rclk=1,Physical therapy clinic,PULASKI+COUNTY+ARKANSAS,Physical+Thrapy+Clinic
3,Pinnacle Physical Therapy,https://www.google.com/maps/place/Pinnacle+Physical+Therapy/data=!4m6!3m5!1s0x87d2a07784ae93bb:0x56fb839ef6e92302!8m2!3d34.797294!4d-92.413297!16s%2Fg%2F11gb3mkx_q?authuser=0&hl=en&rclk=1,Physical therapy clinic,PULASKI+COUNTY+ARKANSAS,Physical+Thrapy+Clinic
4,Blue Rose Physical Therapy,https://www.google.com/maps/place/Blue+Rose+Physical+Therapy/data=!4m6!3m5!1s0xabefe1e4d0190995:0x93c8eacb1abc46e0!8m2!3d34.8017438!4d-92.2551351!16s%2Fg%2F11rrm_w38p?authuser=0&hl=en&rclk=1,Physical therapy clinic,PULASKI+COUNTY+ARKANSAS,Physical+Thrapy+Clinic


In [6]:
search_string = search_string.replace('+'," ").capitalize()
data["Business"] = data["Business"].str.capitalize()

In [7]:
print(data["Business"])
print(search_string)

0      Physical therapy clinic
1      Physical therapy clinic
2      Physical therapy clinic
3      Physical therapy clinic
4      Physical therapy clinic
                ...           
164          Orthopedic clinic
165          Orthopedic clinic
166          Orthopedic clinic
167          Orthopedic clinic
168          Orthopedic clinic
Name: Business, Length: 169, dtype: object
Physical thrapy clinic


In [8]:
data.shape

(169, 5)

In [9]:
# Obtain business information for each individual location
for i in range(len(data)):
    t = pd.Timestamp.now()
    url = data.loc[i, "LocHREF"]#"https://www.google.com/maps/place/CORA+Physical+Therapy+South+County/data=!4m6!3m5!1s0x87d8cf35f7d3f769:0x7b3019c994319873!8m2!3d38.5039686!4d-90.3788281!16s%2Fg%2F1pp2tvx8w?authuser=0&hl=en&rclk=1"
    driver.get(url)
    time.sleep(1)
    try:
        Name = driver.find_element_by_xpath('//*[@id="QA0Szd"]/div/div/div[1]/div[2]/div/div[1]/div/div/div[2]/div[1]/div[1]/div[1]/h1/span[1]').text
        data.loc[i,"Name"] = Name
    except:
        pass
    
    try:
        Business_b = driver.find_element_by_xpath('//*[@id="QA0Szd"]/div/div/div[1]/div[2]/div/div[1]/div/div/div[2]/div[1]/div[1]/div[2]/div/div[2]/span[1]/span[1]/button').text
        data.loc[i,"Business_b"] = Business_b
    except:
        pass
    
    try:
        Address  = driver.find_element_by_xpath('//button[contains(@ aria-label,"Address")]').text
        data.loc[i,"Address"] = Address
    except:
        pass
    
    try:
        AddressDetail = driver.find_element_by_xpath('//div[contains(@ aria-label,"Located in")]').text
        data.loc[i,"AddressDetail"] = AddressDetail
    except:
        pass
    
    try:
        WebSite = driver.find_element_by_xpath('//a[contains(@ aria-label,"Website")]').text
        data.loc[i,"WebSite"] = WebSite
    except:
        pass
    
    print(i,"\t",t, data.loc[i,"LocName"],Name, Business, Address , WebSite)    
    data.to_csv("google_map_result_{}_{}_B.csv".format(prefix,search_string),index = False)
    
data.head(5)

0 	 2022-07-04 23:38:43.745494 UAMS Health - Physical Therapy and Occupational Therapy Center UAMS Health - Physical Therapy and Occupational Therapy Center Orthopedic clinic 501 Jack Stephens Drive Jackson T. Stephens Spine and Neuroscience Institute Level B, Little Rock, AR 72205 uamshealth.com
1 	 2022-07-04 23:39:02.000596 MCH Physical Therapy MCH Physical Therapy Orthopedic clinic 1 Treasure Hill Rd #2219, Little Rock, AR 72205 mchphysicaltherapy.com
2 	 2022-07-04 23:39:05.266346 Physical Therapy Advantage LLC Physical Therapy Advantage LLC Orthopedic clinic 6 Seven Acres Dr ste a, Little Rock, AR 72223 ptadvantagear.com
3 	 2022-07-04 23:39:07.563854 Pinnacle Physical Therapy Pinnacle Physical Therapy Orthopedic clinic 12511 Cantrell Rd Suite 100, Little Rock, AR 72223 pinnacledpt.com
4 	 2022-07-04 23:39:10.057379 Blue Rose Physical Therapy Blue Rose Physical Therapy Orthopedic clinic 5013 John F Kennedy Blvd, North Little Rock, AR 72116 bluerosephysicaltherapy.com
5 	 2022-07-

46 	 2022-07-04 23:40:59.202073 Orthopedic Rehabilitation Orthopedic Rehabilitation Orthopedic clinic 5601 John F Kennedy Blvd, North Little Rock, AR 72116 uamshealth.com
47 	 2022-07-04 23:41:01.679305 Integrative Rehabilitation Services Integrative Rehabilitation Services Orthopedic clinic 12621 Valleywood Dr, Little Rock, AR 72211 uamshealth.com
48 	 2022-07-04 23:41:04.332078 Therapy Zone Therapy Zone Orthopedic clinic 101 Morgan Keegan Dr B, Little Rock, AR 72202 therapyzoneinc.com
49 	 2022-07-04 23:41:07.274234 Troy W. Alberson, MSPT Troy W. Alberson, MSPT Orthopedic clinic 119 W H Ave, North Little Rock, AR 72116 therapyzoneinc.com
50 	 2022-07-04 23:41:09.627771 Physical Therapy Institute Physical Therapy Institute Orthopedic clinic 5307 Kavanaugh Blvd, Little Rock, AR 72207 ptipt.com
51 	 2022-07-04 23:41:11.999565 Athletes In Pain Sports Therapy Clinic Athletes In Pain Sports Therapy Clinic Orthopedic clinic 1121 S Bowman Rd c3, Little Rock, AR 72211 athletesinpain.com
52 	 

93 	 2022-07-04 23:43:54.287246 Baptist Health Therapy Center-Saddle Creek Baptist Health Therapy Center-Saddle Creek Orthopedic clinic 5900 Ranch Dr, Little Rock, AR 72223 baptist-health.com
94 	 2022-07-04 23:43:56.789809 Baptist Health Rehabilitation Institute-Little Rock Baptist Health Rehabilitation Institute-Little Rock Orthopedic clinic 9501 Baptist Health Dr, Little Rock, AR 72205 baptist-health.com
95 	 2022-07-04 23:43:59.395220 Kelley Glenn Kelley Glenn Orthopedic clinic 6 Seven Acres Dr Ste A, Little Rock, AR 72223 ptadvantagear.com
96 	 2022-07-04 23:44:01.445229 UAMS Health - Orthopaedic Clinic UAMS Health - Orthopaedic Clinic Orthopedic clinic 600 Autumn Rd, Little Rock, AR 72211 uamshealth.com
97 	 2022-07-04 23:44:03.879131 Mentor Network Mentor Network Orthopedic clinic 4500 W Commercial Dr, North Little Rock, AR 72116 uamshealth.com
98 	 2022-07-04 23:44:06.200265 Omnis Rehab: Joint and Performance Center Omnis Rehab: Joint and Performance Center Orthopedic clinic 12

141 	 2022-07-04 23:45:52.261729 Harris and Renshaw Physical Therapy - Sherwood Harris and Renshaw Physical Therapy - Sherwood Orthopedic clinic 13133 AR-107, Sherwood, AR 72120 harrisandrenshawpt.com
142 	 2022-07-04 23:45:54.405297 Egoscue of Little Rock Egoscue of Little Rock Orthopedic clinic 14001 Chenal Pkwy Ste 2, Little Rock, AR 72211 egoscue.com
143 	 2022-07-04 23:45:56.910109 UAMS Health - Family Medical Center UAMS Health - Family Medical Center Orthopedic clinic 521 Jack Stephens Drive First floor, Little Rock, AR 72205 uamshealth.com
144 	 2022-07-04 23:45:59.417550 Pain Care Associates Pain Care Associates Orthopedic clinic 8801 W Markham St, Little Rock, AR 72205 paincarelr.com
145 	 2022-07-04 23:46:02.902714 Concentra Urgent Care Concentra Urgent Care Orthopedic clinic 3470 Landers Rd, North Little Rock, AR 72117 concentra.com
146 	 2022-07-04 23:46:05.389073 UAMS Health - STRIVE Clinic UAMS Health - STRIVE Clinic Orthopedic clinic 4701 Fairway Ave, North Little Rock,

,LocName,LocHREF,Business,raw text,search string,Name,Business_b,Address,WebSite
0,UAMS Health - Physical Therapy and Occupational Therapy Center,https://www.google.com/maps/place/UAMS+Health+-+Physical+Therapy+and+Occupational+Therapy+Center/data=!4m6!3m5!1s0x87d2a4b1aaaaaaab:0xe81e8e3d113da9b6!8m2!3d34.7473395!4d-92.318282!16s%2Fg%2F1vyxjyxq?authuser=0&hl=en&rclk=1,Physical therapy clinic,PULASKI+COUNTY+ARKANSAS,Physical+Thrapy+Clinic,UAMS Health - Physical Therapy and Occupational Therapy Center,Physical therapy clinic,"501 Jack Stephens Drive Jackson T. Stephens Spine and Neuroscience Institute Level B, Little Rock, AR 72205",uamshealth.com
1,MCH Physical Therapy,https://www.google.com/maps/place/MCH+Physical+Therapy/data=!4m6!3m5!1s0x87d2a3fd6cd599fd:0x27c5eaacd1a6be7!8m2!3d34.7599151!4d-92.3801111!16s%2Fg%2F1w456483?authuser=0&hl=en&rclk=1,Physical therapy clinic,PULASKI+COUNTY+ARKANSAS,Physical+Thrapy+Clinic,MCH Physical Therapy,Physical therapist,"1 Treasure Hill Rd #2219, Little Rock, AR 72205",mchphysicaltherapy.com
2,Physical Therapy Advantage LLC,https://www.google.com/maps/place/Physical+Therapy+Advantage+LLC/data=!4m6!3m5!1s0x87cd5f61629df399:0x9220fab6450d8a06!8m2!3d34.8014426!4d-92.4488647!16s%2Fg%2F1th0bzpn?authuser=0&hl=en&rclk=1,Physical therapy clinic,PULASKI+COUNTY+ARKANSAS,Physical+Thrapy+Clinic,Physical Therapy Advantage LLC,Physical therapy clinic,"6 Seven Acres Dr ste a, Little Rock, AR 72223",ptadvantagear.com
3,Pinnacle Physical Therapy,https://www.google.com/maps/place/Pinnacle+Physical+Therapy/data=!4m6!3m5!1s0x87d2a07784ae93bb:0x56fb839ef6e92302!8m2!3d34.797294!4d-92.413297!16s%2Fg%2F11gb3mkx_q?authuser=0&hl=en&rclk=1,Physical therapy clinic,PULASKI+COUNTY+ARKANSAS,Physical+Thrapy+Clinic,Pinnacle Physical Therapy,Physical therapy clinic,"12511 Cantrell Rd Suite 100, Little Rock, AR 72223",pinnacledpt.com
4,Blue Rose Physical Therapy,https://www.google.com/maps/place/Blue+Rose+Physical+Therapy/data=!4m6!3m5!1s0xabefe1e4d0190995:0x93c8eacb1abc46e0!8m2!3d34.8017438!4d-92.2551351!16s%2Fg%2F11rrm_w38p?authuser=0&hl=en&rclk=1,Physical therapy clinic,PULASKI+COUNTY+ARKANSAS,Physical+Thrapy+Clinic,Blue Rose Physical Therapy,Physical therapy clinic,"5013 John F Kennedy Blvd, North Little Rock, AR 72116",bluerosephysicaltherapy.com


In [ ]:
# Obtain latitude and longitude for each Location
url = 'https://www.mapdevelopers.com/geocode_tool.php'
driver.get(url)
for i in range(len(data)):
    t = pd.Timestamp.now()
    url = 'https://www.mapdevelopers.com/geocode_tool.php'
    x = data.loc[i, 'Address']
    try:
        what_search_box = driver.find_element_by_id("address")
        what_search_box.clear()
        what_search_box.send_keys(x)
        what_search_box.send_keys(Keys.ENTER)
        time.sleep(2)    

        data.loc[i,"Lat"] = driver.find_element_by_id('display_lat').text
        data.loc[i,"Long"] = driver.find_element_by_id('display_lng').text
    except:
        pass
    print(i,pd.Timestamp.now() - t, data.loc[i,['Lat','Long']])
    data.to_csv("google_map_result_{}_{}_c.csv".format(prefix,search_string),index = False)
#     print(i, df['zipcode'], t, pd.Timestamp.now() - t)

0 0 days 00:00:02.189663 Lat      34.746604
Long    -92.318702
Name: 0, dtype: object
1 0 days 00:00:02.114096 Lat       34.76098
Long    -92.376324
Name: 1, dtype: object
2 0 days 00:00:02.118590 Lat      34.8025926
Long    -92.4481446
Name: 2, dtype: object
3 0 days 00:00:02.125588 Lat      34.797705
Long    -92.412872
Name: 3, dtype: object
4 0 days 00:00:02.144171 Lat      34.8005552
Long    -92.2551476
Name: 4, dtype: object
5 0 days 00:00:02.133298 Lat      34.7934443
Long    -92.2128483
Name: 5, dtype: object
6 0 days 00:00:02.158699 Lat      34.755572
Long    -92.329364
Name: 6, dtype: object
7 0 days 00:00:02.141801 Lat      34.769051
Long    -92.405743
Name: 7, dtype: object
8 0 days 00:00:02.140800 Lat      34.7542555
Long    -92.3898166
Name: 8, dtype: object
9 0 days 00:00:02.143188 Lat      34.796137
Long    -92.235534
Name: 9, dtype: object
10 0 days 00:00:02.137256 Lat     34.7949147
Long    -92.223912
Name: 10, dtype: object
11 0 days 00:00:02.142563 Lat      34.769071

93 0 days 00:00:02.112144 Lat      34.8081495
Long    -92.4629905
Name: 93, dtype: object
94 0 days 00:00:02.116699 Lat          34.745
Long    -92.3794444
Name: 94, dtype: object
95 0 days 00:00:02.129497 Lat      34.8025926
Long    -92.4481446
Name: 95, dtype: object
96 0 days 00:00:02.100334 Lat       34.749128
Long    -92.4028372
Name: 96, dtype: object
97 0 days 00:00:02.097463 Lat      34.797341
Long    -92.224012
Name: 97, dtype: object
98 0 days 00:00:02.097848 Lat     34.7103716
Long    -92.410692
Name: 98, dtype: object
99 0 days 00:00:02.098540 Lat      34.7934443
Long    -92.2128483
Name: 99, dtype: object
100 0 days 00:00:02.129123 Lat      34.7482467
Long    -92.3198885
Name: 100, dtype: object
101 0 days 00:00:02.110994 Lat      34.797341
Long    -92.224012
Name: 101, dtype: object
102 0 days 00:00:02.089182 Lat      34.7903426
Long    -92.4018974
Name: 102, dtype: object
103 0 days 00:00:02.100370 Lat      34.7903426
Long    -92.4018974
Name: 103, dtype: object
104 0 da

In [ ]:
data['ZipCode']=data['Address'].str[-5:].str.strip().astype(int)
data['State']  =data['Address'].str[-8:-6]
data

In [ ]:
county = pd.read_excel(r'C:\Users\Dshahabi\Documents\SampleWorks\HMSA\2019ZipMkt.xlsx')
county["zipcode"]=county["zipcode"].astype(int)
county = county.set_index("zipcode")['County'].to_dict()
data["County"] = data["ZipCode"].map(county)


In [ ]:
data

In [ ]:
data.to_csv("google_map_result_{}_{}_d.csv".format(prefix,search_string),index = False)


In [ ]:
End!